In [5]:
!pip install deims
!pip install xmltodict
!pip install json2xml
!pip install dict2xml
!pip install dicttoxml


In [18]:
import uuid
import json
import xmltodict
import deims
import datetime
import os
from json2xml import json2xml
from json2xml.utils import readfromurl, readfromstring, readfromjson
import xml.etree.ElementTree as ET
import dicttoxml

# add a responsible party to an xml contact element
def xml_add_contact(gmd_contact, name, organisation, email):
    gmd_responsibleparty = ET.SubElement(gmd_contact, 'gmd:CI_ResponsibleParty')
    gmd_individualname = ET.SubElement(gmd_responsibleparty, 'gmd:individualName')
    gmd_individualname_string = ET.SubElement(gmd_individualname, 'gco:CharacterString')
    gmd_individualname_string.set('xmlns:gco', 'http://www.isotc211.org/2005/gco')
    gmd_individualname_string.text = name
    gmd_role = ET.SubElement(gmd_responsibleparty, 'gmd:role')
    gmd_role_code = ET.SubElement(gmd_role, 'gmd:CI_RoleCode')
    gmd_role_code.set('codeList', 'http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#CI_RoleCode')
    gmd_role_code.set('codeListValue', 'pointOfContact')
        
    gmd_organisationName = ET.SubElement(gmd_responsibleparty, 'gmd:organisationName')
    gmd_organisationName_string = ET.SubElement(gmd_organisationName, 'gco:CharacterString')
    gmd_organisationName_string.set('xmlns:gco', 'http://www.isotc211.org/2005/gco')
    gmd_organisationName_string.text = organisation
        
    gmd_contactinfo = ET.SubElement(gmd_responsibleparty, 'gmd:contactInfo')
    gmd_CI_Contact = ET.SubElement(gmd_contactinfo, 'gmd:CI_Contact')
    gmd_address = ET.SubElement(gmd_CI_Contact, 'gmd:address')
    gmd_CI_address = ET.SubElement(gmd_address, 'gmd:CI_Address')
    gmd_mailaddress = ET.SubElement(gmd_CI_address, 'gmd:electronicMailAddress')
    gmd_mailaddress_string = ET.SubElement(gmd_mailaddress, 'gco:CharacterString')
    gmd_mailaddress_string.set('xmlns:gco', 'http://www.isotc211.org/2005/gco')
    gmd_mailaddress_string.text = email

In [20]:
with open('getcapabilities_1.3.0.xml') as fd:
    doc = xmltodict.parse(fd.read())

for layer in doc['WMS_Capabilities']['Capability']['Layer']['Layer']:
    root = ET.Element('MD_Metadata')
    root.set('xmlns:gmd', 'http://www.isotc211.org/2005/gmd')
    root.set('xmlns:srv', 'http://www.isotc211.org/2005/srv')
    root.set('xmlns:gco', 'http://www.isotc211.org/2005/gco')
    root.set('xmlns:xlink', 'http://www.w3.org/1999/xlink')
    root.set('xmlns:sos', 'http://www.opengis.net/sos/1.0')
    root.set('xmlns:xs', 'http://www.w3.org/2001/XMLSchema')
    root.set('xmlns:gmx', 'http://www.isotc211.org/2005/gmx')
    root.set('xmlns:gts', 'http://www.isotc211.org/2005/gts')
    root.set('xmlns:ows', 'http://www.opengis.net/ows/1.1')
    root.set('xmlns:gml', 'http://www.opengis.net/gml')
    root.set('xmlns:xsi', 'http://www.w3.org/2001/XMLSchema-instance')
    root.set('xsi:schemaLocation', 'http://www.isotc211.org/2005/gmd http://schemas.opengis.net/csw/2.0.2/profiles/apiso/1.0.0/apiso.xsd')

    record_uuid = str(uuid.uuid4())
    gmd_fileIdentifier = ET.SubElement(root, 'gmd:fileIdentifier')
    gmd_uuid = ET.SubElement(gmd_fileIdentifier, 'gco:CharacterString')
    gmd_uuid.text = record_uuid
    gmd_language = ET.SubElement(root, 'gmd:language')
    gmd_language_eng = ET.SubElement(gmd_language, 'gco:CharacterString')
    gmd_language_eng.text = 'eng'
    gmd_characterSet = ET.SubElement(root, 'gmd:characterSet')
    gmd_characterSetCode = ET.SubElement(gmd_characterSet, 'gmd:MD_CharacterSetCode')
    gmd_characterSetCode.text = "MD_CharacterSetCode_utf8"
    gmd_characterSetCode.set('codeListValue', 'MD_CharacterSetCode_utf8')
    gmd_characterSetCode.set('codeList', 'http://www.isotc211.org/2005/resources/Codelist/gmxCodelists.xml#MD_CharacterSetCode')
    gmd_characterSetCode.set('codeSpace', 'ISOTC211/19115')
    gmd_hierarchylevel = ET.SubElement(root, 'gmd:hierarchyLevel')
    gmd_MD_ScopeCode = ET.SubElement(gmd_hierarchylevel, 'gmd:hierarchyLevel')
    gmd_MD_ScopeCode.set('codeList', 'http://www.isotc211.org/2005/resources/Codelist/gmxCodelists.xml#MD_ScopeCode')
    gmd_MD_ScopeCode.set('codeListValue', 'dataset')
    gmd_contact = ET.SubElement(root, 'gmd:contact')
    
    # date stamp
    
     # fetch EPSG code
    epsg_code = ''
    for value in layer['CRS']:
        if "EPSG:" in value:
            epsg_code = value[5:]
            
    gmd_referenceSystemInfo = ET.SubElement(root, 'gmd:referenceSystemInfo')
    gmd_MD_ReferenceSystem = ET.SubElement(gmd_referenceSystemInfo, 'gmd:MD_ReferenceSystem')
    gmd_referenceSystemIdentifier = ET.SubElement(gmd_MD_ReferenceSystem, 'gmd:referenceSystemIdentifier')
    gmd_RS_Identifier = ET.SubElement(gmd_referenceSystemIdentifier, 'gmd:RS_Identifier')
    gmd_RS_Identifier_code = ET.SubElement(gmd_RS_Identifier, 'gmd:code')
    gmd_RS_Identifier_gmx_anchor = ET.SubElement(gmd_RS_Identifier_code, 'gmx:Anchor')
    gmd_RS_Identifier_gmx_anchor.set('xmlns:xlink', 'http://www.w3.org/1999/xlink')
    gmd_RS_Identifier_gmx_anchor.set('xlink:href', 'http://www.opengis.net/def/crs/EPSG/0/4258')
    gmd_RS_Identifier_gmx_anchor.text = 'EPSG:' + epsg_code

    keywords_to_be_printed = []
    authors = None
    # process keywords for site information and temporal extent
    for keyword in layer['KeywordList']['Keyword']:
        uri = None

        if "project: " in keyword:
            if "e-shape" in keyword:
                uri = "https://doi.org/10.3030/820852"
                keyword = "e-shape"

            if "AGAME" in keyword:
                keyword = "AGAME"

        if "variable: " in keyword:
            data_product_type = keyword[10:]
            if data_product_type == "Phenology":
                keyword = "Phenology"
                uri = "https://vocabs.lter-europe.net/EnvThes/21647"
                xml_add_contact(gmd_contact, "Saverio Vicario", "CNR", "saverio.vicario@iia.cnr.it")
                
            if data_product_type == "Hydroperiod":
                keyword = "Hydroperiod"
                xml_add_contact(gmd_contact, "Ioannis Manakos", "CERTH", "imanakos@iti.gr")
                
            if data_product_type == "GPP":
                keyword = "GPP"
                uri = "http://vocabs.lter-europe.net/EnvThes/21074"
                xml_add_contact(gmd_contact, "Mario Alberto Fruentes-Monjarez", "Deltares", "mario.fuentesmonjaraz@deltares.nl")
                xml_add_contact(gmd_contact, "Valeria Mobilia", "Deltares", "valeria.mobilia@deltares.nl")

            if data_product_type == "snowcover":
                uri = "https://vocabs.lter-europe.net/EnvThes/21559"
                keyword = "snowCover"
                xml_add_contact(gmd_contact, "Maria Adama", "CNR", "adamo@iia.cnr.it")
                xml_add_contact(gmd_contact, "Chiara Richiardi", "CNR", "chiara.richiardi@iia.cnr.it")

        if "site: " in keyword:
            deims_suffix = keyword[6:]
            #print(site_record)
            uri = "https://deims.org/" + deims_suffix
            site_record = deims.getSiteById(deims_suffix)

            try:
              keyword = site_record["title"]
            except:
              print(layer['Title'])
              print(deims_suffix)

        if "https://doi.org/" in keyword:
            doi = {
                "name": "B2Share",
                "description": "Download data on B2Share",
                "url": keyword
            }

        if "time: " in keyword:
            time_string = keyword[6:]

            if "-" in time_string:
                time_values = time_string.split('-')
                period = {
                    "begin": time_values[0] + '-01-01',
                    "end": time_values[1] + '-01-01'
                }

            else:
                # might be necessary to have start/end date with days and months, not just year
                period = {"begin": time_string}

        if uri:
            formatted_keyword = {
                "value": keyword,
                "uri": uri
            }
        else:
            formatted_keyword = {
                "value": keyword
            }

        keywords_to_be_printed.append(formatted_keyword)

    #print(deims_suffix)

   

    # boundaries
    westBoundLongitude = layer['EX_GeographicBoundingBox']["westBoundLongitude"]
    eastBoundLongitude = layer['EX_GeographicBoundingBox']["eastBoundLongitude"]
    southBoundLatitude = layer['EX_GeographicBoundingBox']["southBoundLatitude"]
    northBoundLatitude = layer['EX_GeographicBoundingBox']["northBoundLatitude"]

    record = {
        "id": record_uuid,
        "uri": "https://catalogue.lter-europe.net/id/" + record_uuid,
        "type": "signpost",
        "title": layer['Title'],
        "description": layer['Abstract'], # we could add a sentence that the metadata record was generated automatically
        "metadataDate": str(datetime.datetime.now()),
        "resourceIdentifiers": {"code": "TBD/" + record_uuid},
        "descriptiveKeywords": [{"keywords": keywords_to_be_printed}],
        "responsibleParties": authors,
        "temporalExtents": period,
        "boundingBoxes": [
             {
              "westBoundLongitude": westBoundLongitude,
              "eastBoundLongitude": eastBoundLongitude,
              "southBoundLatitude": southBoundLatitude,
              "northBoundLatitude": northBoundLatitude,
              "bounds":  {
                    "type": "Feature",
                    "properties": {},
                    "geometry": {
                        "type": "Polygon",
                        "coordinates": [
                            [
                                [westBoundLongitude, southBoundLatitude],
                                [westBoundLongitude, northBoundLatitude],
                                [eastBoundLongitude, northBoundLatitude],
                                [eastBoundLongitude, southBoundLatitude],
                                [westBoundLongitude, southBoundLatitude]
                            ]
                        ]
                    }
                },
              "coordinates": f"[[[{westBoundLongitude}, {southBoundLatitude}], [{westBoundLongitude}, {northBoundLatitude}], [{eastBoundLongitude}, {northBoundLatitude}], [{eastBoundLongitude}, {southBoundLatitude}], [{westBoundLongitude}, {southBoundLatitude}]]]"
            }
        ],
        "supplemental": [doi],
        "service": {
            "type": "WMS"
        },
        "mapDataDefinition": {
            "data": [
                {
                    "path": "https://elter.datalabs.ceh.ac.uk/geoserver/ows?version=1.3.0",
                    "epsgCode": epsg_code,
                    "type": "POLYGON",
                    "features": {
                        "name": layer['Name'],
                        "label": layer['Title'],
                        "style": {
                            "colour": "#000000"
                        }
                    },
                    "bytetype": False
              }
            ]
        },
        "resourceType": {
            "value": "signpost"
        },
        "notGEMINI": False,
        "deimsSites": [{
            "title": site_record["title"],
            "id": deims_suffix,
            "url": "https://deims.org/" + deims_suffix
        }],
        "linkedDocument": False,
        "mapViewable": False,
        "incomingCitationCount": 0,
        "authors": authors
    }

    filename = record_uuid + ".xml"
    base_dir = os.getcwd() + '/files_for_csw'
    destination_path = os.path.join(base_dir, filename)
    
    # Convert a JSON to XML
    xml_object = json2xml.Json2xml(record).to_xml()
    xml = dicttoxml.dicttoxml(record, return_bytes=False, attr_type=False, include_encoding=False, custom_root='gmd:MD_Metadata')
    
    #with open(destination_path, "w+") as f:
    #    f.write(xml)
    
    tree = ET.ElementTree(root)
    ET.indent(tree, space="\t", level=0)
    tree.write(destination_path, encoding="utf-8",  xml_declaration=True)


KeyboardInterrupt: 